37-element Vector{Vector{Real}}:
 [1, 1, 2, 0.0922, 0.047, 10, 0]
 [2, 2, 3, 0.493, 0.2511, 10, 0]
 [3, 3, 4, 0.366, 0.1864, 10, 0]
 [4, 4, 5, 0.3811, 0.1941, 10, 0]
 [5, 5, 6, 0.819, 0.707, 10, 0]
 [6, 6, 7, 0.1872, 0.6188, 10, 0]
 [7, 7, 8, 1.7114, 1.2351, 10, 0]
 [8, 8, 9, 1.03, 0.74, 10, 0]
 [9, 9, 10, 1.044, 0.74, 10, 0]
 [10, 10, 11, 0.1966, 0.065, 10, 0]
 ⋮
 [29, 29, 30, 0.5075, 0.2585, 10, 0]
 [30, 30, 31, 0.9744, 0.963, 10, 0]
 [31, 31, 32, 0.3105, 0.3619, 10, 0]
 [32, 32, 33, 0.341, 0.5302, 10, 0]
 [33, 8, 21, 2, 2, 10, 1]
 [34, 9, 15, 2, 2, 10, 1]
 [35, 12, 22, 2, 2, 10, 1]
 [36, 18, 33, 0.5, 0.5, 10, 1]
 [37, 25, 29, 0.5, 0.5, 10, 1]

In [1]:
function MakeMask(x, y, list)
    Mask = zeros(x, y)
    for i = 1:x
        Mask[i, findall(x -> x == i, list)] .= 1
    end
    return Mask
end

function MakeIncMatrix(s, t)
    MaxNode = max(maximum(s), maximum(t))
    I = zeros(MaxNode, length(s))
    for j = 1:length(s)
        I[s[j], j] = 1
        I[t[j], j] = -1
    end
    return I
end

MakeIncMatrix (generic function with 1 method)

In [2]:
Sb = 100                   # MW/MVar/MVA
Vb = 12.66                # kV
Zb = Vb^2/Sb              # O
Ib = Sb/(sqrt(3)*Vb)      # kA
V0 = 1                    # p.u.
V_max = 1.05
V_min = 0.95
NT = 1                     # 在这里设置步数 step模式为1
Big_M_FF = 40             # 单商品流的BigM
Big_M_V = 3               # 压降松弛的BigM
BigM_SC = 2
include("33bw_data.jl")

N_Line = size(DN_Branch_Data, 1)
N_TL = 5                    # 需要增加tieline辨识
N_NL = 32 
N_Bus = 33
N_PL = N_Bus - 1
Pd_Data = DN_Bus_Data[:, 2] ./ Sb
Pd_Data_all = sum(Pd_Data)
Qd_Data = DN_Bus_Data[:, 3] ./ Sb
Qd_Data_all = sum(Qd_Data)
Pd_ratio = Pd_Data ./ Pd_Data_all    # Active load ratio
Qd_ratio = Qd_Data ./ Qd_Data_all    # Reactive load ratio

N_Branch = size(DN_Branch_Data, 1)
Branch_start = DN_Branch_Data[:, 2]   # 每条线路的起点和终点
Branch_end = DN_Branch_Data[:, 3]
LS_Mask = MakeMask(N_Bus, N_Branch, Branch_start)
LE_Mask = MakeMask(N_Bus, N_Branch, Branch_end)
pIn = MakeIncMatrix(Branch_start, Branch_end)   # 潮流方程专用  关联矩阵，node-branch incidence matrix  jk-ij
pInn = copy(pIn)
pInn[pInn .> 0] .= 0    # 潮流方程专用 Inn is the negative part of I   +ij
R_Branch0 = DN_Branch_Data[:, 4] ./ Zb   # 线路阻抗
X_Branch0 = DN_Branch_Data[:, 5] ./ Zb   # 线路电抗
SZ_Branch0 = R_Branch0 .^ 2 + X_Branch0 .^ 2   # SOCP模型专用 线路阻抗模平方
S_Branch0 = DN_Branch_Data[:, 6] ./ Sb
Alive = DN_Branch_Data[:, 7] 

N_DG = size(DG_Data, 1)
DataDN_IndDG = DG_Data[:, 2]   # DG接入位置
DataDN_IndBSDG = DG_Data[findall(x -> x == 1, DG_Data[:, 7]), 2]   # DG中黑启动机组
DataDN_IndNMDG = DG_Data[findall(x -> x == 0, DG_Data[:, 7]), 2]   # DG中非黑启动机组
N_BSDG = size(DataDN_IndBSDG, 1)
N_NMDG = size(DataDN_IndNMDG, 1)
P_DG_max0 = DG_Data[:, 3] ./ Sb
P_DG_min0 = DG_Data[:, 4] ./ Sb
Q_DG_max0 = DG_Data[:, 5] ./ Sb
Q_DG_min0 = DG_Data[:, 6] ./ Sb
DG_Mask = MakeMask(N_Bus, N_DG, DataDN_IndDG)   # DG接入关联矩阵
BSDG_Mask = MakeMask(N_Bus, N_BSDG, DataDN_IndBSDG)   # BSDG接入关联矩阵
NMDG_Mask = MakeMask(N_Bus, N_NMDG, DataDN_IndNMDG)   # Normal DG接入关联矩阵

BoundsError: BoundsError: attempt to access 33-element Vector{Vector{Real}} at index [1:33, 2]